# Telco data analysis

In [1]:
import pandas as pd
import numpy as np

## Loading data

In [2]:
data_path = 'TelcoData.csv'
data = pd.read_csv(
    data_path,
    sep=';',
    dtype={'StudentID': object, 'SenderID': object, 'ReceiverID': object, 'EventType': object}
)

In [3]:
data.head()

,EventTime,StudentID,SenderID,ReceiverID,EventType,EventLength
0,2011-08-06 04:00:35,62560,176422,62560,1,9
1,2011-08-06 04:28:00,62560,176422,62560,1,31
2,2011-08-07 19:35:55,62560,176422,62560,1,100
3,2011-08-07 20:33:52,62560,176422,62560,1,153
4,2011-08-07 20:34:02,62560,176422,62560,1,32


## Data summary

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7575865 entries, 0 to 7575864
Data columns (total 6 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   EventTime    object
 1   StudentID    object
 2   SenderID     object
 3   ReceiverID   object
 4   EventType    object
 5   EventLength  int64 
dtypes: int64(1), object(5)
memory usage: 346.8+ MB


In [18]:
data.describe(include='all').transpose()

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
EventTime,7575865,6859244,2012-12-25 05:28:08,50,NaN,NaN,NaN,NaN,NaN,NaN,NaN
StudentID,7575865,218,51102,137742,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SenderID,7575865,23351,121,78886,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ReceiverID,7575865,30917,51102,76968,NaN,NaN,NaN,NaN,NaN,NaN,NaN
EventType,7575865,2,1,7096844,NaN,NaN,NaN,NaN,NaN,NaN,NaN
EventLength,7575865.0,NaN,NaN,NaN,56.345224,134.764799,0.0,17.0,35.0,67.0,26649.0


## Data cleaning

### Remove unwanted senders and recievers

In [5]:
data.drop(data[~(data['SenderID'].map(len) == 5) | ~(data['ReceiverID'].map(len) == 5)].index, inplace=True)
data.shape

(537575, 6)

In [6]:
data.head()

,EventTime,StudentID,SenderID,ReceiverID,EventType,EventLength
41,2011-08-15 20:03:31,20905,20905,13116,0,15
42,2011-08-15 20:03:37,13116,20905,13116,0,0
52,2011-08-15 20:47:40,75178,75178,30952,0,39
53,2011-08-15 20:47:50,30952,75178,30952,0,23
54,2011-08-15 20:48:52,75178,75178,60830,0,14


### Remove duplicates

In [7]:
time_delta = pd.Timedelta('15 s')

In [8]:
new_data = data.copy()

In [9]:
new_data['EventTime'] = pd.to_datetime(new_data['EventTime'])

In [10]:
# Remove rows with analogous StudentID, SenderID and RecieverID, that are moved in time maxiamally by 15 seconds.
def is_duplicate(row, df):
    duplicates = df[
        (df['EventTime'] >= row['EventTime'] - time_delta) & 
        (df['EventTime'] < row['EventTime']) &
        (df['SenderID'] == row['SenderID']) &
        (df['ReceiverID'] == row['ReceiverID']) &
        (df['SenderID'] == row['SenderID'])
    ]
    return len(duplicates) > 0

In [98]:
new_data2 = new_data[new_data['EventTime'] < dti].copy()

In [12]:
%time new_data['IsDuplicate'] = new_data.apply(lambda row: is_duplicate(row, new_data), axis=1)

Wall time: 17h 40min 46s


In [99]:
%time new_data2['IsDuplicate'] = new_data2.apply(lambda row: is_duplicate(row, new_data2), axis=1)

Wall time: 48min 24s


In [121]:
new_data2 = pd.read_csv(
    'TelcoData_before-2012_cleaned_marked-duplicates.csv',
    sep=';',
    dtype={'StudentID': object, 'SenderID': object, 'ReceiverID': object, 'EventType': object, 'IsDuplicate': bool}
)

In [14]:
new_data.shape

(537575, 7)

In [15]:
new_data[new_data['IsDuplicate'] == True].shape

(153997, 7)

In [16]:
new_data.drop(new_data[new_data['IsDuplicate'] == True].index, inplace=True)

In [18]:
new_data['StudentID'].nunique()

214

In [19]:
for sid in new_data['StudentID'].unique():
    new_data[new_data['StudentID'] == sid][['SenderID', 'ReceiverID', 'EventTime']].to_csv('stud/telcodata-' + sid + '.csv', sep=';', index=False)

In [139]:
new_data[new_data['StudentID'] == '10060'][['SenderID', 'ReceiverID', 'EventTime']].to_csv('stud/telcodata-10060.csv', sep=';', index=False)

In [137]:
del new_data2['IsDuplicate']

In [13]:
new_data.to_csv('TelcoData_cleaned_marked-duplicates.csv', sep=';', index=False)

## Clean data summary

In [6]:
data.describe(include='all').transpose()

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
EventTime,537575,521130,2011-10-18 19:04:30,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN
StudentID,537575,214,68494,37964,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SenderID,537575,229,13799,29268,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ReceiverID,537575,229,68494,33425,NaN,NaN,NaN,NaN,NaN,NaN,NaN
EventType,537575,2,1,503221,NaN,NaN,NaN,NaN,NaN,NaN,NaN
EventLength,537575.0,NaN,NaN,NaN,51.030976,103.28941,0.0,17.0,34.0,65.0,17052.0


In [92]:
data.tail(5)

,EventTime,StudentID,SenderID,ReceiverID,EventType,EventLength
7575581,2014-08-21 21:14:48,78911,51722,78911,1,21
7575582,2014-08-21 21:15:46,78911,78911,51722,1,6
7575640,2014-08-21 23:52:42,98453,98453,86352,0,29
7575642,2014-08-22 00:20:50,78911,51722,78911,1,5
7575643,2014-08-22 00:23:55,78911,78911,51722,1,24


## Saving clean telco data

In [93]:
# todo: code

# Input data generation

## Model parameters

ForgettingType;Days;Mu;Theta;Lambda;Unit  
exponential;3;0.3;0.2;0.00563145983483561;3600

In [77]:
rows = [('exponential', 3, 0.3, 0.2, 0.00563145983483561, 3600)]
df = pd.DataFrame(rows, columns =['ForgettingType', 'Days', 'Mu', 'Theta', 'Lambda', 'Unit'])
df.to_csv('modelparameters.csv', sep=';', index=False)

## Survey dates

StudentID;SurveyNo;CompletionDate  
10060;2;2012-01-29 21:35:00

In [91]:
rows = []

fst_date = new_data['EventTime'][41]
last_date = new_data['EventTime'][7575643]
nr_days = (last_date - fst_date).days + 1
dates = [fst_date + pd.Timedelta(str(i+1)+' d') for i in range(nr_days)]

student_ids = set(data['SenderID'])

for s in student_ids:
    for i, d in enumerate(dates):
        rows.append((s, i+1, d))
        
df = pd.DataFrame(rows, columns =['StudentID', 'SurveyNo', 'CompletionDate'])
df.to_csv('surveydates.csv', sep=';', index=False)

# Insights


## Nr events in time

In [90]:
# todo: plot it nicely
new_data['EventTime'].groupby([new_data.EventTime.dt.year, new_data.EventTime.dt.month]).agg('count')

EventTime  EventTime
2011       8            14063
           9            27384
           10           23010
           11           20661
           12           20019
2012       1            20438
           2            24946
           3            25102
           4            21412
           5            21346
           6            14028
           7            16553
           8            19755
           9            21791
           10           17567
           11           19549
           12           18347
2013       1            16475
           2            14939
           3            14159
           4            13323
           5            12381
           6             7032
           7             9893
           8            14799
           9             8606
           10            6301
           11            7219
           12            8666
2014       1            11445
           2            11025
           3            11409
           4       